In [6]:
def command_line(args):
    from subprocess import Popen, PIPE
    """
    just include the arguments as a list
    """
    process = Popen(args, stdout=PIPE, stderr=PIPE)
    stdout, stderr = process.communicate()
    return stdout,stderr
def upload_session():
    stdout,stderr = command_line(['rclone','copy','upload','AuroraGoogleDrive:AuroraProject/sessions'])
upload_session()

b''
b''
